In [1]:
from ess.loki.nurf import utils, fluo, plot
from ess.loki.nurf import ill_auxilliary_funcs as ill 

# standard library imports
import itertools
import os
from typing import Optional, Type, Union

# related third party imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.gridspec as gridspec
from IPython.display import display, HTML
from scipy.optimize import leastsq  # needed for fitting of turbidity

# local application imports
import scippnexus as snx
import scipp as sc
from scipp.ndimage import median_filter



In [3]:
# Prepare for export to .dat for uv and fluo

# path to LOKI-like files
process_folder='/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/UU/ILL_947_June_2021/212/d22/exp_9-13-947/processed/ess_version'
# change to folder
os.chdir(process_folder)

# export path for .dat files
path_output='/Users/gudlo523/Library/CloudStorage/OneDrive-LundUniversity/UU/ILL_947_June_2021/212/d22/exp_9-13-947/processed/ess_version/dat-files'

# experimental data sets
exp5= [66017, 66020, 66023, 66026, 66029, 66032, 66034, 66037, 66040, 66043, 66046]
exp6= [66050, 66053, 66056, 66059, 66062, 66065, 66068, 66071, 66074, 66077, 66080]

exp2= [65925, 65927, 65930, 65933, 65936, 65939, 65942, 65945, 65948, 65951, 65954, 65957]
exp3= [65962, 65965, 65968, 65971, 65974, 65977, 65980, 65983, 65986, 65989, 65992]

exp7= [66083, 66086, 66089, 66092, 66095, 66098, 66101, 66104, 66107, 66110, 66113]
exp8= [66116, 66119, 66122, 66125, 66128, 66131, 66134, 66137, 66140, 66143, 66146]

Cell below shows how to load fluo spectra from a Loki.nxs file

In [4]:
name='066017.nxs'
exp_meth='fluorescence'

# load a file and caluclate fluo spectra within a LoKI.nxs file
fluo_dict=utils.load_nurfloki_file(name,exp_meth)
fluo_da=fluo.normalize_fluo(**fluo_dict)

print("This is the resulting dataarray")
display(fluo_da)

# or in one line, preserves source attribute
fluo_da2=fluo.load_and_normalize_fluo(name)

display(fluo_da2)


<scipp.DataArray>
Dimensions: Sizes[spectrum:12, wavelength:1044, ]
Coordinates:
  integration_time          float32            [µs]  (spectrum)  [5e+07, 5e+07, ..., 5e+07, 5e+07]
  is_dark                      bool        <no unit>  (spectrum)  [False, False, ..., False, False]
  is_data                      bool        <no unit>  (spectrum)  [True, True, ..., True, True]
  is_reference                 bool        <no unit>  (spectrum)  [False, False, ..., False, False]
  monowavelengths           float32             [nm]  (spectrum)  [295, 295, ..., 320, 320]
  time                        int64            [µs]  (spectrum)  [1655992257000000, 1655992257000000, ..., 1655992257000000, 1655992257000000]
  wavelength                float32             [nm]  (wavelength)  [248.684, 249.485, ..., 1042.31, 1043.03]
Data:
                            float32         [counts]  (spectrum, wavelength)  [3.218e-05, 3.212e-05, ..., 3.202e-05, 3.196e-05]
Attributes:
  source                     string        <no unit>  (spectrum)  ["066017.nxs", "066017.nxs", ..., "066017.nxs", "066017.nxs"]

<scipp.DataArray>
Dimensions: Sizes[wavelength:1044, ]
Coordinates:
  wavelength                float32             [nm]  (wavelength)  [248.684, 249.485, ..., 1042.31, 1043.03]
Data:
                            float32         [counts]  (wavelength)  [3.226e-05, 3.202e-05, ..., 3.198e-05, 3.196e-05]
Attributes:
  integration_time          float32            [µs]  ()  [5e+07]
  is_dark                      bool        <no unit>  ()  [True]
  is_data                      bool        <no unit>  ()  [False]
  is_reference                 bool        <no unit>  ()  [False]
  monowavelengths           float32             [nm]  ()  [nan]
  source                     string        <no unit>  ()  ["066017.nxs"]
  time                        int64            [µs]  ()  [1655992257000000]

<scipp.DataArray>
Dimensions: Sizes[wavelength:1044, ]
Coordinates:
  wavelength                float32             [nm]  (wavelength)  [248.684, 249.485, ..., 1042.31, 1043.03]
Data:
                            float32         [counts]  (wavelength)  [3.212e-05, 3.2e-05, ..., 3.19e-05, 3.188e-05]
Attributes:
  integration_time          float32            [µs]  ()  [5e+07]
  is_dark                      bool        <no unit>  ()  [False]
  is_data                      bool        <no unit>  ()  [False]
  is_reference                 bool        <no unit>  ()  [True]
  monowavelengths           float32             [nm]  ()  [nan]
  source                     string        <no unit>  ()  ["066017.nxs"]
  time                        int64            [µs]  ()  [1655992257000000]

This is the resulting dataarray


<scipp.DataArray>
Dimensions: Sizes[spectrum:12, wavelength:1044, ]
Coordinates:
  integration_time          float32            [µs]  (spectrum)  [5e+07, 5e+07, ..., 5e+07, 5e+07]
  is_dark                      bool        <no unit>  (spectrum)  [False, False, ..., False, False]
  is_data                      bool        <no unit>  (spectrum)  [True, True, ..., True, True]
  is_reference                 bool        <no unit>  (spectrum)  [False, False, ..., False, False]
  monowavelengths           float32             [nm]  (spectrum)  [295, 295, ..., 320, 320]
  time                        int64            [µs]  (spectrum)  [1655992257000000, 1655992257000000, ..., 1655992257000000, 1655992257000000]
  wavelength                float32             [nm]  (wavelength)  [248.684, 249.485, ..., 1042.31, 1043.03]
Data:
                            float32         [counts]  (spectrum, wavelength)  [6.00012e-08, 1.19999e-07, ..., 1.20002e-07, 7.99992e-08]

<scipp.DataArray>
Dimensions: Sizes[spectrum:12, wavelength:1044, ]
Coordinates:
  integration_time          float32            [µs]  (spectrum)  [5e+07, 5e+07, ..., 5e+07, 5e+07]
  is_dark                      bool        <no unit>  (spectrum)  [False, False, ..., False, False]
  is_data                      bool        <no unit>  (spectrum)  [True, True, ..., True, True]
  is_reference                 bool        <no unit>  (spectrum)  [False, False, ..., False, False]
  monowavelengths           float32             [nm]  (spectrum)  [295, 295, ..., 320, 320]
  time                        int64            [µs]  (spectrum)  [1655992257000000, 1655992257000000, ..., 1655992257000000, 1655992257000000]
  wavelength                float32             [nm]  (wavelength)  [248.684, 249.485, ..., 1042.31, 1043.03]
Data:
                            float32         [counts]  (spectrum, wavelength)  [3.218e-05, 3.212e-05, ..., 3.202e-05, 3.196e-05]
Attributes:
  source                     string        <no unit>  (spectrum)  ["066017.nxs", "066017.nxs", ..., "066017.nxs", "066017.nxs"]

<scipp.DataArray>
Dimensions: Sizes[wavelength:1044, ]
Coordinates:
  wavelength                float32             [nm]  (wavelength)  [248.684, 249.485, ..., 1042.31, 1043.03]
Data:
                            float32         [counts]  (wavelength)  [3.226e-05, 3.202e-05, ..., 3.198e-05, 3.196e-05]
Attributes:
  integration_time          float32            [µs]  ()  [5e+07]
  is_dark                      bool        <no unit>  ()  [True]
  is_data                      bool        <no unit>  ()  [False]
  is_reference                 bool        <no unit>  ()  [False]
  monowavelengths           float32             [nm]  ()  [nan]
  source                     string        <no unit>  ()  ["066017.nxs"]
  time                        int64            [µs]  ()  [1655992257000000]

<scipp.DataArray>
Dimensions: Sizes[wavelength:1044, ]
Coordinates:
  wavelength                float32             [nm]  (wavelength)  [248.684, 249.485, ..., 1042.31, 1043.03]
Data:
                            float32         [counts]  (wavelength)  [3.212e-05, 3.2e-05, ..., 3.19e-05, 3.188e-05]
Attributes:
  integration_time          float32            [µs]  ()  [5e+07]
  is_dark                      bool        <no unit>  ()  [False]
  is_data                      bool        <no unit>  ()  [False]
  is_reference                 bool        <no unit>  ()  [True]
  monowavelengths           float32             [nm]  ()  [nan]
  source                     string        <no unit>  ()  ["066017.nxs"]
  time                        int64            [µs]  ()  [1655992257000000]

<scipp.DataArray>
Dimensions: Sizes[spectrum:12, wavelength:1044, ]
Coordinates:
  integration_time          float32            [µs]  (spectrum)  [5e+07, 5e+07, ..., 5e+07, 5e+07]
  is_dark                      bool        <no unit>  (spectrum)  [False, False, ..., False, False]
  is_data                      bool        <no unit>  (spectrum)  [True, True, ..., True, True]
  is_reference                 bool        <no unit>  (spectrum)  [False, False, ..., False, False]
  monowavelengths           float32             [nm]  (spectrum)  [295, 295, ..., 320, 320]
  time                        int64            [µs]  (spectrum)  [1655992257000000, 1655992257000000, ..., 1655992257000000, 1655992257000000]
  wavelength                float32             [nm]  (wavelength)  [248.684, 249.485, ..., 1042.31, 1043.03]
Data:
                            float32         [counts]  (spectrum, wavelength)  [6.00012e-08, 1.19999e-07, ..., 1.20002e-07, 7.99992e-08]
Attributes:
  source                     string        <no unit>  (spectrum)  ["066017.nxs", "066017.nxs", ..., "066017.nxs", "066017.nxs"]

The cell below shows how to plot the fluo content of a Loki file. Currently, the code differentiates between good and bad spectra because during the measurements at ILL technical problems occured.

In [ ]:
#How to plot fluo the fluo spectra contained in one LoKI.nxs file
plot.plot_fluo(name)

In [ ]:
#How to load and normalize fluo spectra of one file in one go.
fluo_da=fluo.load_and_normalize_fluo(name) 
display(fluo_da)

How to apply a median filter. Let's use the new median_filter offered by scipp.

In [ ]:
#How to load and normalize fluo spectra of one file in one go.
fluo_da=fluo.load_and_normalize_fluo(name) 

# prepare kernel
kernel_size=15

# apply median filter
fluo_da_filt=utils.nur





fluo_da_filt=median_filter(fluo_da, size=kernel_size_scipp)
#show effect of median_filter
legend_props = {"show": True, "loc": 1}
fig1=sc.plot(sc.collapse(fluo_da, keep='wavelength'), marker='.', title='before any median filter',legend=legend_props)
fig2=sc.plot(sc.collapse(fluo_da_filt, keep='wavelength'), marker='.', title=f'after median filter - scipp, size={kernel_size_scipp} ',legend=legend_props)
display(fig1,fig2)







In [ ]:
fluo_peak_int(
    fluo_da: sc.DataArray,
    wllim=None,
    wulim=None,
    wl_unit=None,
    medfilter=True,
    kernel_size=None,
):

w
fluo.fluo_peak_int